In [3]:
#from pyscopus import Scopus
import json
import os
import numpy as np
#import requests
    
## Load configuration
# filename = os.getcwd() + "/elsapy_/config.json"
# with open(filename) as f:
#     config = json.load(f)

# scopus = Scopus(config['apikey'])

In [4]:
import pickle
#import os

#import data from pickle file
name = str(os.getcwd()) + '/search_df.pkl'
with open(name, 'rb') as infile:
    search_df = pickle.load(infile)

In [5]:
import pandas as pd
import nltk
import gensim

#load Google's word2vec model. This model is a large pre-trained model for converting words to vectors
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/Volumes/MasterDrive/GoogleNews-vectors-negative300.bin', binary=True)

/Users/alexhuang/anaconda3/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [6]:
#download these files once
#nltk.download('punkt')
#nltk.download('stopwords')

#load stop words, words to delete from the files
stop = set(nltk.corpus.stopwords.words('english'))

# nonempty full texts
full_text = search_df['abstract']



#convert text to token list
def entry_vectorize(entry):
    full_text_entry = nltk.word_tokenize(str(entry))
    full_text_vector = [word_vectors[w.lower()] for w in full_text_entry if w.lower() in word_vectors.vocab and w.lower() not in stop]
    return np.array(full_text_vector)

full_text_vectorized = full_text.apply(lambda entry: entry_vectorize(entry))

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
#full_text_vectorized.shape

def get_doc_word_length(doc):
    return doc.shape[0]

#def pad(doc):

doc_lengths = full_text_vectorized.apply(lambda doc: get_doc_word_length(doc))
zero_mask = (doc_lengths >= 10)

search_df_new = search_df[zero_mask]

full_text_vec_nonzero = full_text_vectorized[zero_mask]
doc_lengths = full_text_vec_nonzero.apply(lambda doc: get_doc_word_length(doc))

# Try without outliers first
full_text_vec_new = full_text_vec_nonzero



# outlier_mask = abs(doc_lengths - (np.mean(doc_lengths))) < 3*np.std(doc_lengths)
# full_text_vec_new = full_text_vec_nonzero[outlier_mask]
# new_max_words = np.max(doc_lengths[outlier_mask])
#print(full_text_vec_no_outliers.shape)


# mask = (outlier_mask & zero_mask)
# full_text_vec_new = full_text_vectorized[mask]
# print(full_text_vec_new.shape)




# print(search_df_new.shape)

#print(doc_lengths[outlier_mask].mean())

# plt.figure()
# plt.hist(search_df_new['citation_count'])
# plt.show()


In [8]:
#linear regression model
import sklearn
from sklearn.linear_model import LogisticRegression

y = search_df_new['citation_count'] #[pd.notnull(search_df['full_text'])]
X = full_text_vec_new

doc_lengths_new = X.apply(lambda doc: get_doc_word_length(doc))

max_words = np.max(doc_lengths_new)
# print(max_words)
# print(np.mean(doc_lengths_new))
# plt.hist(doc_lengths,bins=20)
# print(np.std(doc_lengths_new))

def pad(x):
    u_x = x.mean(axis=0)
    to_pad = np.repeat(u_x, max_words - x.shape[0]).reshape((max_words - x.shape[0],x.shape[1]))
    padded_x = np.concatenate([x,to_pad])
    return padded_x
    
X_padded = np.concatenate(X.apply(lambda x: pad(x)).to_list()).reshape((X.shape[0],max_words,X[0].shape[1]))




In [9]:
X_padded.shape
#test if there are any citations, so make y binary
y_binary = np.where(y > 0, 1, 0)

X_pathname = "/Volumes/MasterDrive/ML project data/X.pkl"
with open(X_pathname, 'wb') as output_X:
    pickle.dump(X_padded, output_X, pickle.HIGHEST_PROTOCOL)
    
y_pathname = "/Volumes/MasterDrive/ML project data/y.pkl"
with open(y_pathname, 'wb') as output_y:
    pickle.dump(y_binary, output_y, pickle.HIGHEST_PROTOCOL)

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA



# Xtr, Xts, ytr, yts = train_test_split(X_padded, y_binary, test_size = 0.2)

# tr_shape = ((Xtr.shape[0],Xtr.shape[1]*Xtr.shape[2]))
# ts_shape = ((Xts.shape[0],Xts.shape[1]*Xts.shape[2]))

# scaler = StandardScaler()
# Xtr_scaled = scaler.fit_transform(Xtr.reshape(tr_shape))
# Xts_scaled = scaler.transform(Xts.reshape(ts_shape))

# pca = PCA()
# Xtr_reduced = pca.fit_transform(Xtr.reshape(tr_shape))
# Xts_reduced = pca.transform(Xts.reshape(ts_shape))





In [ ]:
# from sklearn.metrics import f1_score
# from sklearn.linear_model import LinearRegression

# model = LinearRegression()
# model.fit(Xtr.reshape(tr_shape), ytr)

# yhat = model.predict(Xts.reshape(ts_shape))

# # F1 score is more appropriate due to imbalanced dataset
# print(f1_score(yts,yhat))

In [ ]:
# zero_citation_dates = pd.to_datetime(search_df_new['cover_date'][search_df_new['citation_count']==0]).sort_values()
                                     
# nonzero_citation_dates = pd.to_datetime(search_df_new['cover_date'][search_df_new['citation_count']!=0]).sort_values()

# mean_date_zero = (np.array(zero_citation_dates, dtype='datetime64[s]')
#         .view('i8')
#         .mean()
#         .astype('datetime64[s]'))

# mean_date_nonzero = (np.array(nonzero_citation_dates, dtype='datetime64[s]')
#         .view('i8')
#         .mean()
#         .astype('datetime64[s]'))